#### Imports/setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ee
import os
# import pandas as pd
import geemap
import time
import functools
import os
import sys
# from datetime import datetime

import requests # may not be needed
from sidecar import Sidecar

from modules.gee_initialize import initialize_ee 
initialize_ee()

from parameters.config_runtime import (
    geo_id_column, 
    out_csv, 
    out_shapefile,
    cols_ind_1_treecover,
    cols_ind_2_commodities,
    cols_ind_3_dist_before_2020,
    cols_ind_4_dist_after_2020,
)

from modules.agstack_to_gee import (
    start_agstack_session,
    get_agstack_token,
    register_fc_and_append_to_csv,
    # add_geo_ids_to_csv_from_lookup_df,
    add_geo_ids_to_csv_from_lookup_csv,
    add_empty_column_to_csv,
    remove_column_from_csv
)

# from modules.stats import get_stats,reformat_whisp_fc,get_stats_formatted

from parameters.config_asr_url_info import asset_registry_base, user_registry_base

from parameters.config_asr_credentials import email, password

from modules.utils import (
    collection_properties_to_df,
    remove_geometry_from_feature_collection,
    get_centroid
)

from modules.tidy_tables import whisp_risk, add_eudr_risk_col, add_indicators, select_years_in_range, create_wildcard_column_list

def whisp_stats_as_df (roi): 
    %store roi
    %run process_whisp_stats.ipynb   
    return df_out
   

Earth Engine has been initialized with the specified credentials.


#### Set up asset registry

In [3]:
session = start_agstack_session(email,password,user_registry_base)

token = get_agstack_token(email, password, asset_registry_base)

#### Inputs : without GEOIDS

GEE asset (feature collection) 

In [4]:
example_plot_feature_col_asset = 'projects/ee-andyarnellgee/assets/p0004_commodity_mapper_support/seg_civ_PCBM'
# example_plot_feature_col_asset = 'projects/ee-cocoacmr/assets/feature_data/cicc_polys_2k'
# Define the fraction of features you want to sample (e.g., 0.5 for 50%)
sampleFraction = 0.0030

In [5]:
fc = ee.FeatureCollection(example_plot_feature_col_asset)

randomColumn = 'random'
fc_r = fc.randomColumn(randomColumn,4)

# Filter the feature collection based on the random column to get the sample
fc_sample = fc_r.filter(ee.Filter.lt(randomColumn, sampleFraction));

roi = fc_sample

print(roi.size().getInfo())

roi


16


In [13]:
# temp code: uncomment below line if want to test using point inputs based on polygon centroid

# roi = roi.map(lambda feature: ee.Feature(feature.geometry().centroid(1))) 

### Whisp it
- If over >500 polygons currently this will go to Google Drive and require manual uploading here
- Stats sent to a separate notebook 'process_whisp_stats.pynb'

In [6]:
df = whisp_stats_as_df(roi)

Stored 'roi' (FeatureCollection)
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Datasets present in lookup:  True
geo id not present ['Area_ha', 'Geometry_type', 'Country', 'lat', 'lon']
...stats processing started
...ee to df processing time: 5.0629496574401855 seconds


### Display it


In [7]:
df

,Area_ha,Geometry_type,Country,lat,lon,EUFO_2020,GLAD_Primary,TMF_undist,JAXA_FNF_2020,GFC_TC_2020,...,TMF_loss_before_2020,GFC_loss_before_2020,ESA_fire_before_2020,MODIS_fire_before_2020,TMF_deg_after_2020,TMF_loss_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,system:index
0,0.01,Point,CIV,-3.695531,6.108556,0.009878,0,0.000000,0.009878,0.009878,...,0,0.000000,0,0,0.000000,0,0,0,0,000000000000000007b6
1,0.01,Point,CIV,-3.649126,6.038068,0.009879,0,0.000000,0.009879,0.009879,...,0,0.000000,0,0,0.000000,0,0,0,0,00000000000000000121
2,0.01,Point,CIV,-3.611461,6.122915,0.009878,0,0.000000,0.009878,0.009878,...,0,0.000000,0,0,0.000000,0,0,0,0,000000000000000010e2
3,0.01,Point,CIV,-3.656310,6.110293,0.009878,0,0.009878,0.009878,0.009878,...,0,0.000000,0,0,0.009878,0,0,0,0,0000000000000000023e
4,0.01,Point,CIV,-3.667554,6.085371,0.009879,0,0.000000,0.009879,0.009879,...,0,0.000000,0,0,0.000000,0,0,0,0,000000000000000003d6
5,0.01,Point,CIV,-3.589833,6.113249,0.009878,0,0.009878,0.009878,0.009878,...,0,0.000000,0,0,0.000000,0,0,0,0,00000000000000000df1
6,0.01,Point,CIV,-3.653701,6.115954,0.009878,0,0.000000,0.009878,0.009878,...,0,0.000000,0,0,0.000000,0,0,0,0,000000000000000001ca
7,0.01,Point,CIV,-3.611783,6.106373,0.009878,0,0.000000,0.009878,0.009878,...,0,0.000000,0,0,0.000000,0,0,0,0,0000000000000000110f
8,0.01,Point,CIV,-3.659884,6.030822,0.009880,0,0.009880,0.009880,0.009880,...,0,0.000000,0,0,0.000000,0,0,0,0,0000000000000000029d
9,0.01,Point,CIV,-3.699359,6.101030,0.009878,0,0.009878,0.009878,0.009878,...,0,0.000000,0,0,0.000000,0,0,0,0,00000000000000000835


#### Export it to CSV file
NB skip if exporting to Google Drive

In [8]:
#export output csv
print  (out_csv)
df.to_csv(path_or_buf=out_csv,header=True,index=False)

print ("output csv: ", out_csv)


/home/sepal-user/whisp/whisp_output_table.csv
output csv:  /home/sepal-user/whisp/whisp_output_table.csv


#### Add Geo ids
##### Step 1. Register polygons:  compiles geo_ids in a lookup csv
- NB registration takes a long time if many polygons (a few seconds each one). AgStack will update this in future. 
- In the meantime to stop timeouts extend session length click dollar sign and edit under 'Sessions'.
- If processing doesn't complete, rerun and it should pickup from where you got to.

In [9]:
register_fc_and_append_to_csv(
    feature_col=roi,
    geo_id_column="Geo_id",
    output_lookup_csv="temp_geo_id_lookup.csv",
    join_id_column="system:index",
    token=token,
    session=session,
    asset_registry_base=asset_registry_base,
    override_checks=False,
    remove_temp_csv=False,
    debug=True)


Check passed: feature collection and table same size: 16 rows
Number without geo ids:0. 
 Processing started...
Backup file copied successfully to: backup_csvs/temp_geo_id_lookup_20240513_141915.csv
Done


##### Step 2. Join geo ids from lookup csv to Whisp stats csv

In [10]:
add_geo_ids_to_csv_from_lookup_csv(
        input_csv="whisp_output_table.csv",
        geo_id_lookup_csv="temp_geo_id_lookup.csv",
        join_id_column="system:index",
        geo_id_column=geo_id_column,
        # override_checks=False, # needs implementing
        overwrite=False,
        drop_geo=False,
        debug=True)

new csv:  copy_whisp_output_table.csv


##### Optional: remove "system:index" column 
NB this is needed for joining geo_ids to csv (from lookup table). Check you have all your geo_ids first and if in doubt run on a copy of the results


In [11]:
# remove_column_from_csv(
#     csv_file="copy_whisp_output_table.csv", # this may change depending on if overwrite is on
#     column_name="system:index"
# )

##### Calculate EUDR risk category

In [12]:
# Replace with the name of your CSV file
csv_file ='copy_whisp_output_table.csv'

# import as a dataframe
df_w_stats = pd.read_csv(csv_file)


df_w_risk = whisp_risk(
    df = df_w_stats, 
    ind_1_pcent_threshold=10,
    ind_2_pcent_threshold=10,
    ind_3_pcent_threshold=0,
    ind_4_pcent_threshold=0
    )

output_file='copy_whisp_output_table_w_risk.csv'

# export to a csv
df_w_risk.to_csv(output_file)


# display df with added risk column
df_w_risk


,Area_ha,Geo_id,Geometry_type,Country,lat,lon,EUFO_2020,GLAD_Primary,TMF_undist,JAXA_FNF_2020,...,TMF_loss_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,system:index,Indicator_1_treecover,Indicator_2_commodities,Indicator_3_disturbance_before_2020,Indicator_4_disturbance_after_2020,EUDR_risk
0,0.01,eb7183ae1c62b147e099de79265607bf1a53cb4ff717d5...,Point,CIV,-3.695531,6.108556,0.009878,0,0.000000,0.009878,...,0,0,0,0,000000000000000007b6,yes,yes,yes,no,low
1,0.01,37485ae8d1dd2d047caa0fbc57a72e508e83b50bd3c4c1...,Point,CIV,-3.649126,6.038068,0.009879,0,0.000000,0.009879,...,0,0,0,0,00000000000000000121,yes,no,yes,no,low
2,0.01,4b1ad3cd5dae97e1617b8cc7b39d2b5557f318abd8f47e...,Point,CIV,-3.611461,6.122915,0.009878,0,0.000000,0.009878,...,0,0,0,0,000000000000000010e2,yes,yes,yes,no,low
3,0.01,407e41306ce7edb11a28a3810c2db6b40fd0a4ab2d2e9a...,Point,CIV,-3.656310,6.110293,0.009878,0,0.009878,0.009878,...,0,0,0,0,0000000000000000023e,yes,no,no,yes,high
4,0.01,049f0050aef223e4facfd31abd0b86de6f8181a2d76d87...,Point,CIV,-3.667554,6.085371,0.009879,0,0.000000,0.009879,...,0,0,0,0,000000000000000003d6,yes,no,no,no,more_info_needed
5,0.01,375ba910a586a125ff451e960451990123b31ba0e54fcd...,Point,CIV,-3.589833,6.113249,0.009878,0,0.009878,0.009878,...,0,0,0,0,00000000000000000df1,yes,yes,no,no,low
6,0.01,ed21ceabff267e7017b346b4a0afa47f29af1177b41194...,Point,CIV,-3.653701,6.115954,0.009878,0,0.000000,0.009878,...,0,0,0,0,000000000000000001ca,yes,no,yes,no,low
7,0.01,a56bc7919b0a7935552df735c1db96d0fb41202df4d944...,Point,CIV,-3.611783,6.106373,0.009878,0,0.000000,0.009878,...,0,0,0,0,0000000000000000110f,yes,no,yes,no,low
8,0.01,146d51ab4e6a931f392efde0b270a3865e6d59a8eea559...,Point,CIV,-3.659884,6.030822,0.009880,0,0.009880,0.009880,...,0,0,0,0,0000000000000000029d,yes,no,no,no,more_info_needed
9,0.01,b98c8bc02d5057ad4f2a1ceea4fa235af0d1333703ffae...,Point,CIV,-3.699359,6.101030,0.009878,0,0.009878,0.009878,...,0,0,0,0,00000000000000000835,yes,no,no,no,more_info_needed
